In [2]:
# http://numericinsight.com/uploads/A_Gentle_Introduction_to_Backpropagation.pdf
#
# creating a vanilla Neural Network (MLP) from scratch
#
# structure: 1 x 3 x 1
# 1 inputs ---- 1 hidden layer with 3 units ---- 1 outputs

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import sys

In [4]:
def ground_prediction(x1_i, x2_i):
    ground = np.bitwise_or(x1_i, x2_i)
    return ground
    
    
n_samples = 1000
X1 = np.random.choice(np.random.randint(0, 10000, 10000), n_samples)
X2 = np.random.choice(np.random.randint(10000, 20000, 10000), n_samples)
X1 = np.random.choice(np.random.randint(0, 2, 10000), n_samples)
X2 = np.random.choice(np.random.randint(0, 2, 10000), n_samples)
X = np.asarray([X1, X2]).transpose()
X_norm = X

# scaler = MinMaxScaler(feature_range=(-1, 1))
# scaler.fit(X)
# X_norm = scaler.transform(X)

y = list()
for i in np.arange(X.shape[0]):
    y_i = ground_prediction(X[i,0], X[i,1])
    y.append(y_i)
    
y = np.asarray(y) 
y_norm = y

# y = y.reshape(-1,1)
# scaler_y = MinMaxScaler(feature_range=(-1, 1))
# scaler_y.fit(y)
# y_norm = scaler_y.transform(y)

In [5]:
print X_norm[1:15]
print y_norm[1:15]

[[1 0]
 [0 0]
 [0 0]
 [0 1]
 [1 0]
 [1 0]
 [1 1]
 [0 1]
 [1 1]
 [0 1]
 [0 1]
 [1 0]
 [1 1]
 [1 0]]
[1 0 0 1 1 1 1 1 1 1 1 1 1 1]


In [7]:
# Initialize Weight Layers
def initialize_layers():
    # +1 represents bias purposes
    X_feat_n = X.shape[1]
    L1_units_n = X_feat_n
    L2_units_n = 3
    L3_units_n = 1
    
    L1_weights = np.random.randn(L2_units_n*(L1_units_n + 1))
    L1_weights = L1_weights.reshape(L2_units_n, (L1_units_n + 1))

    L2_weights = np.random.randn(L3_units_n, (L2_units_n + 1))
    L2_weights = L2_weights.reshape(L3_units_n, (L2_units_n + 1))

    return L2_weights, L1_weights


def sigmoid_fun(z):
    activation = 1 / (1 + np.exp(-z))
    return activation
 

def sigmoid_fun_derivative(a):
    derv = np.multiply(a, (1.0 - a))
    return derv
    
    
def add_bias(a_matrix):
    a_matrix = np.row_stack((np.ones(a_matrix.shape[1]), a_matrix))
    return a_matrix


def compute_mean_squared_error(a_3, idx):
    y_3 = y_norm[idx].reshape(a_3.shape)
    err = 0.5*np.power(y_3 - a_3, 2)
    
    err = np.sum(err)/batch_size
    return err


def log_likelihood(a_3, idx):
    y_3 = y_norm[idx].reshape(a_3.shape)
    
    ll = np.multiply(y_3, np.log(a_3)) + np.multiply(1 - y_3, np.log(1 - a_3))
    ll = np.sum(ll)/batch_size
    
    return ll
    

In [37]:
def compute_backpropagation_errors(L2_weights, a_3, a_2, idx):    
    #d_3 = a_3 - y_norm[idx].reshape(a_3.shape)
    d_3 = np.multiply((a_3 - y_norm[idx].reshape(a_3.shape)), sigmoid_fun_derivative(a_3))
    
    # remove the bias column of weights and its activation
    d_2 = np.multiply(L2_weights[:, 1:].transpose().dot(d_3), sigmoid_fun_derivative(a_2[1:]))
    
    return d_3, d_2


def compute_gradient(d, a):
    grad = d.dot(a.transpose()) / batch_size
    return grad
    

def update_weights(L2_weights, L1_weights, a_3, a_2, a_1, idx, check_grad=False):
    #import pdb; pdb.set_trace()
    d_3, d_2 = compute_backpropagation_errors(L2_weights, a_3, a_2, idx)
    
    grad_L2 = compute_gradient(d_3, a_2)
    grad_L1 = compute_gradient(d_2, a_1)
    
    L2_weights_updated = L2_weights - r * grad_L2
    L1_weights_updated = L1_weights - r * grad_L1
    
    if (check_grad):
        check_gradient(L2_weights, grad_L2, L1_weights, grad_L1, idx)
    
    return L2_weights_updated, L1_weights_updated


In [38]:
def compute_gradient_approximation(L2_weights, L1_weights, idx, e):
    # 2nd layer
    L2_weights_gradsAprox = np.zeros(L2_weights.shape)
    for i in np.arange(L2_weights.shape[0]):
        for j in np.arange(L2_weights.shape[1]):
            L2_weights_shift = np.copy(L2_weights)
            
            L2_weights_shift[i,j] = L2_weights[i,j] + e
            a_3, a_2, a_1 = feed_forward_NN(L2_weights_shift, L1_weights, idx)
            err_plus = compute_mean_squared_error(a_3, idx)

            L2_weights_shift[i,j] = L2_weights[i,j] - e
            a_3, a_2, a_1 = feed_forward_NN(L2_weights_shift, L1_weights, idx)
            err_minus = compute_mean_squared_error(a_3, idx)
    
            derv = (err_plus - err_minus)/(2*e)
            L2_weights_gradsAprox[i,j] = derv
            
           
    # 1st layer
    L1_weights_gradsAprox = np.zeros(L1_weights.shape)
    for i in np.arange(L1_weights.shape[0]):
        for j in np.arange(L1_weights.shape[1]):
            L1_weights_shift = np.copy(L1_weights)
            
            L1_weights_shift[i,j] = L1_weights[i,j] + e
            a_3, a_2, a_1 = feed_forward_NN(L2_weights, L1_weights_shift, idx)
            err_plus = compute_mean_squared_error(a_3, idx)

            L1_weights_shift[i,j] = L1_weights[i,j] - e
            a_3, a_2, a_1 = feed_forward_NN(L2_weights, L1_weights_shift, idx)
            err_minus = compute_mean_squared_error(a_3, idx)
    
            derv = (err_plus - err_minus)/(2*e)
            L1_weights_gradsAprox[i,j] = derv
            
            
    return L2_weights_gradsAprox, L1_weights_gradsAprox
    


def check_gradient(L2_weights, grad_L2, L1_weights, grad_L1, idx, e=1e-4):
        
    L2_weights_gradsAprox, L1_weights_gradsAprox = compute_gradient_approximation(L2_weights, L1_weights, idx, e)
    
    L2_grads_check = np.allclose(L2_weights_gradsAprox, grad_L2, atol=1e-3)
    if not (L2_grads_check): sys.exit("Error in L2 gradients values checking ")
    
    L2_grads_check_shape = (L2_weights_gradsAprox.shape == grad_L2.shape)
    if not (L2_grads_check_shape): sys.exit("Error in L2 gradients shape checking ")
        
    L1_grads_check = np.allclose(L1_weights_gradsAprox, grad_L1, atol=1e-2)    
    if not (L1_grads_check): sys.exit("Error in L1 gradients values checking")

    L1_grads_check_shape = (L1_weights_gradsAprox.shape == grad_L1.shape)
    if not (L1_grads_check_shape): sys.exit("Error in L1 gradients shape checking ")
    

In [142]:
class NeuralNet:
    
    def __init__(self):
        self.L2_weights, self.L1_weights = initialize_layers()
        self.err_history = []
        self.ll_history = []
    
        # Activation Layer's States
        self.a_3 = np.array([])
        self.a_2 = np.array([])
        self.a_1 = np.array([])
    
    
    def feed_forward_NN(self, idx, predict=False):
        self.a_1 = X_norm[idx, :].transpose()
        self.a_1 = add_bias(self.a_1)

        z_2 = self.L1_weights.dot(self.a_1)
        self.a_2 = sigmoid_fun(z_2)
        self.a_2 = add_bias(self.a_2)

        z_3 = self.L2_weights.dot(self.a_2)
        #a_3 = z_3
        self.a_3 = sigmoid_fun(z_3)

        
        if predict: 
            return self.a_3        
        
    
    
    def update_weights(self, idx, check_grad=False):
        #import pdb; pdb.set_trace()
        d_3, d_2 = compute_backpropagation_errors(self.L2_weights, self.a_3, self.a_2, idx)

        grad_L2 = compute_gradient(d_3, self.a_2)
        grad_L1 = compute_gradient(d_2, self.a_1)

        self.L2_weights +=  (- r * grad_L2)
        self.L1_weights +=  (- r * grad_L1)

        if (check_grad):
            check_gradient(self.L2_weights, grad_L2, self.L1_weights, grad_L1, idx)

        


    def train(self, itr_n=100, batch_size=32, r=0.3, CG = True):

        for i in np.arange(itr_n):
            idx = np.random.choice(np.arange(X.shape[0]), batch_size)
            self.feed_forward_NN(idx)

            err = compute_mean_squared_error(self.a_3, idx)
            self.err_history.append(err)

            ll = log_likelihood(self.a_3, idx)
            self.ll_history.append(ll)

            if (i == 5): CG = False;        
            self.update_weights(idx, check_grad=CG)
    
    
    def predict(self, idx):
        return self.feed_forward_NN(idx, predict=True)


In [143]:
itr_n = 10
batch_size = 1000
r = 0.3

nn_test = NeuralNet()
nn_test.train(itr_n)

In [144]:
nn_test.predict([100])

array([[ 0.38014235]])

In [ ]:
#plt.plot(ll_history)
plt.plot(err_history)
plt.show()

In [ ]:
#idx = np.random.choice(np.arange(X.shape[0]), 5)
idx = np.arange(y.shape[0])
a_3, a_2, a_1 = feed_forward_NN(L2_weights, L1_weights, idx)

y_demo = y[idx]
#y_demo = y_demo.reshape((idx.shape[0],))
#a_3_demo = scaler_y.inverse_transform(a_3) 
a_3_demo = a_3
a_3_demo = a_3_demo.reshape((idx.shape[0],))
y_diff = y_demo - a_3_demo
y_diff_round = y_demo - np.round(a_3_demo)

# for i in np.arange(y_demo.shape[0]):
#     print int(y_demo[i]), int(a_3_demo[i]), int(y_diff[i])
    
for perc in np.arange(0, 100+10, 10):
    print "perc: %s  /  %s" %(perc, np.percentile(y_diff, perc))
    

In [ ]:
print np.percentile(y_diff_round, np.arange(0,110,10))

In [ ]:
print a_3

In [ ]:
X_sig = np.arange(-7.5,7.5, step=.1)
y_sig = sigmoid_fun(X_sig)
y_sig_der = np.multiply(y_sig, 1 - y_sig)
y_sig_der_2 = np.multiply(np.multiply(y_sig, 1 - y_sig), 1 - 2*y_sig)
#y_sig_der = sigmoid_fun_derivative(y_sig)

plt.plot(X_sig, y_sig)
plt.plot(X_sig, y_sig_der)
plt.plot(X_sig, y_sig_der_2)
plt.show()